## Zadanie 1

In [93]:
from collections.abc import MutableMapping
from random import randrange # used to pick MAD parameters

class MapBase(MutableMapping):
    """Our own abstract base class that includes a nonpublic _Item class."""
    #------------------------------- nested _Item class -------------------------------
    class _Item:
        """Lightweight composite to store key-value pairs as map items."""
        __slots__ = '_key', '_value'
        def __init__(self, k, v):
            self._key = k
            self._value = v
        def __eq__(self, other):
            return self._key == other._key # compare items based on their keys
        def __ne__(self, other):
            return not (self == other) # opposite of __eq__
        def __lt__(self, other):
            return self._key < other._key # compare items based on their keys

class UnsortedTableMap(MapBase):
  """Map implementation using an unordered list."""

  def __init__(self):
    """Create an empty map."""
    self._table = []                              # list of _Item's
  
  def __getitem__(self, k):
    """Return value associated with key k (raise KeyError if not found)."""
    for item in self._table:
      if k == item._key:
        return item._value
    raise KeyError('Key Error: ' + repr(k))

  def __setitem__(self, k, v):
    """Assign value v to key k, overwriting existing value if present."""
    for item in self._table:
      if k == item._key:                          # Found a match:
        item._value = v                           # reassign value
        return                                    # and quit    
    # did not find match for key
    self._table.append(self._Item(k,v))

  def __delitem__(self, k):
    """Remove item associated with key k (raise KeyError if not found)."""
    for j in range(len(self._table)):
      if k == self._table[j]._key:                # Found a match:
        self._table.pop(j)                        # remove item
        return                                    # and quit    
    raise KeyError('Key Error: ' + repr(k))

  def __len__(self):
    """Return number of items in the map."""
    return len(self._table)

  def __iter__(self):                             
    """Generate iteration of the map's keys."""
    for item in self._table:
      yield item._key                             # yield the KEY

class HashMapBase(MapBase):
    """ Abstract base class for map using hash-table with MAD compression.
    Keys must be hashable and non-None.
    """
    def __init__(self, cap=11, p=109345121):
        """Create an empty hash-table map.
        cap initial table size (default 11)
        p positive prime used for MAD (default 109345121)
        """
        self._table = cap * [ None ]
        self._n = 0 # number of entries in the map
        self._prime = p # prime for MAD compression
        self._scale = 1 + randrange(p-1) # scale from 1 to p-1 for MAD
        self._shift = randrange(p) # shift from 0 to p-1 for MAD
    def _hash_function(self, k):
        return (hash(k)*self._scale + self._shift) % self._prime % len(self._table)
    def __len__(self):
        return self._n
    def __getitem__(self, k):
        j = self._hash_function(k)
        return self._bucket_getitem(j, k) # may raise KeyError
    def __setitem__(self, k, v):
        j = self._hash_function(k)

        self._bucket_setitem(j, k, v) # subroutine maintains self._n
        if self._n > len(self._table) // 2: # keep load factor <= 0.5
            self._resize(2 * len(self._table) - 1) # number 2^x - 1 is often prime
    def __delitem__(self, k):
        j = self._hash_function(k)
        self._bucket_delitem(j, k) # may raise KeyError
        self._n -= 1
        
    def _resize(self, c):
        """Resize bucket array to capacity c and rehash all items."""
        old = list(self.items()) # use iteration to record existing items
        self._table = c * [None] # then reset table to desired capacity
        self._n = 0 # n recomputed during subsequent adds
        for (k,v) in old:
            self[k] = v # reinsert old key-value pair

class ChainHashMap(HashMapBase):
    """Hash map implemented with separate chaining for collision resolution."""
    def _bucket_getitem(self, j, k):
        #================================================================================
        #j = collision_hash(j) do zadania 3
        bucket = self._table[j]
        if bucket is None:
            raise KeyError('Key Error: ' + repr(k)) # no match found
        return bucket[k] # may raise KeyError
    
    def _bucket_setitem(self, j, k, v):
        #j = collision_hash(j)
        if self._table[j] is None:
            self._table[j] = UnsortedTableMap() # bucket is new to the table
        oldsize = len(self._table[j])
        self._table[j][k] = v
        if len(self._table[j]) > oldsize: # key was new to the table
            self._n += 1 # increase overall map size
    
    def _bucket_delitem(self, j, k):
        bucket = self._table[j]
        if bucket is None:
            raise KeyError('Key Error: ' + repr(k)) # no match found
        del bucket[k]               # may raise KeyError
    
    def __iter__(self):
        for bucket in self._table:
            if bucket is not None: # a nonempty slot
                for key in bucket:
                    yield key

class ProbeHashMap(HashMapBase):
    """Hash map implemented with linear probing for collision resolution."""
    _AVAIL = object() # sentinal marks locations of previous deletions
    def _is_available(self, j):
        """Return True if index j is available in table."""
        return self._table[j] is None or self._table[j] is ProbeHashMap._AVAIL
    def _find_slot(self, j, k):
        """Search for key k in bucket at index j.
        Return (success, index) tuple, described as follows:
        If match was found, success is True and index denotes its location.
        If no match found, success is False and index denotes first available slot.
        """
        firstAvail = None
        while True:
            if self._is_available(j):
                if firstAvail is None:
                    firstAvail = j # mark this as first avail
                if self._table[j] is None:
                    return (False, firstAvail) # search has failed
            elif k == self._table[j]._key:
                return (True, j) # found a match
            j = (j + 1) % len(self._table) # keep looking (cyclically)
    
    def _bucket_getitem(self, j, k):
        found, s = self._find_slot(j, k)
        if not found:
            raise KeyError('Key Error: ' + repr(k)) # no match found
        return self._table[s]._value
    
    def _bucket_setitem(self, j, k, v):
        found, s = self._find_slot(j, k)
        if not found:
            self._table[s] = self._Item(k,v) # insert new item
            self._n += 1 # size has increased
        else:
            self._table[s]._value = v # overwrite existing
    
    def _bucket_delitem(self, j, k):
        found, s = self._find_slot(j, k)
        if not found:
            raise KeyError('Key Error: ' + repr(k)) # no match found
        self._table[s] = ProbeHashMap._AVAIL # mark as vacated
    
    def __iter__(self):
        for j in range(len(self._table)): # scan entire table
            if not self._is_available(j):
                yield self._table[j]._key


def collision_hash(k):
    
    return 7-(k % 7)

In [87]:
tab_asoc = ChainHashMap(cap = 11, p = 11)
tab_asoc._scale = 3
tab_asoc._shift = 5

tab_asoc[12] = "12"
tab_asoc[44] = "44"
tab_asoc[13] = "13"
tab_asoc[88] = "88"
tab_asoc[23] = "23"
tab_asoc[94] = "94"
tab_asoc[11] = "11"
tab_asoc[39] = "39"
tab_asoc[20] = "20"
tab_asoc[16] = "16"
tab_asoc[5] = "5"

indexes = []
for key in tab_asoc.keys():
    indexes.append([key,tab_asoc._hash_function(key)])
    
print(indexes)




[[13, 0], [94, 1], [39, 1], [44, 5], [88, 5], [11, 5], [12, 8], [23, 8], [16, 9], [5, 9], [20, 10]]


## Zadanie 2

In [86]:
tab_asoc = ProbeHashMap(cap = 11, p = 11)
tab_asoc._scale = 3
tab_asoc._shift = 5

tab_asoc[12] = "12"
tab_asoc[44] = "44"
tab_asoc[13] = "13"
tab_asoc[88] = "88"
tab_asoc[23] = "23"
tab_asoc[94] = "94"
tab_asoc[11] = "11"
tab_asoc[39] = "39"
tab_asoc[20] = "20"
tab_asoc[16] = "16"
tab_asoc[5] = "5"

indexes = []
for key in tab_asoc.keys():
    indexes.append([key,tab_asoc._find_slot(tab_asoc._hash_function(key),key)[1]])
    
print(indexes)

[[13, 0], [94, 1], [39, 2], [44, 5], [88, 6], [11, 7], [12, 8], [23, 9], [20, 10], [16, 11], [5, 12]]


## Zadanie 3

In [95]:
tab_asoc = ChainHashMap(cap = 11, p = 11)
tab_asoc._scale = 3
tab_asoc._shift = 5

tab_asoc[12] = "12"
tab_asoc[44] = "44"
tab_asoc[13] = "13"
tab_asoc[88] = "88"
tab_asoc[23] = "23"
tab_asoc[94] = "94"
tab_asoc[11] = "11"
tab_asoc[39] = "39"
tab_asoc[20] = "20"
tab_asoc[16] = "16"
tab_asoc[5] = "5"

indexes = []
for key in tab_asoc.keys():
    indexes.append([key,collision_hash(tab_asoc._hash_function(key))])
    
print(indexes)

[[44, 2], [88, 2], [11, 2], [20, 4], [16, 5], [5, 5], [12, 6], [23, 6], [94, 6], [39, 6], [13, 7]]


## Zadanie 4

<img src="zad4.jpg">

## Zadanie 5

<img src="zad5_2.jpg">
<img src="zad5_1.jpg">

## Zadanie 6

In [19]:
def sort_name(tab,l,r):
    """Sortuje tablicę tab oraz zwraca index pivota, elementu według którego sortowano"""
    i = l
    pivot = tab[r]
    for j in range(l,r):
        if tab[j] <= pivot:
            tab[i], tab[j] = tab[j], tab[i]
            i += 1
    tab[i],tab[r] = tab[r],tab[i]
    return i

def quick_non_recursive(tab,l,r):
    """Szybkie sortowanie które używa stosu do przechowywania indeksów partycji nieposortowanych"""
    # we will use a stack to manage the indexes of partitions
    size = r - l + 1
    stack = [0] * size
    top = -1

    # we start with the given left and right values
    top += 1
    stack[top] = l
    top += 1
    stack[top] = r
    
    #we sort in bounds of the left and right values given on the stack till nothing is left
    while top >= 0:

        r = stack[top]
        top -= 1
        l = stack[top]
        top -= 1

        pivot = sort_name(tab,l,r)
        #if there are unsorted elements on the left or right of the pivot we add new bounds to the stack
        if pivot+1 < r:
            top += 1
            stack[top] = pivot+1
            top += 1
            stack[top] = r
        
        if pivot-1 > l:
            top += 1
            stack[top] = l
            top += 1
            stack[top] = pivot -1


def quick_in_place(array,l,r):
    """Sortowanie szybkie in-place które używa rekurencji by określać kolejne indexy partycji do posortowania"""
    p = sort_name(array,l,r)
    if p+1 < r:
        quick_in_place(array,p+1,r)
    if p-1 > l:
        quick_in_place(array,l,p-1)





In [22]:
array = [3,7,6,1,2,9,5]
array2 = [7,2,6,10,4,14,8,3,1,5]
quick_non_recursive(array,0,len(array)-1)
quick_in_place(array2,0,len(array2)-1)
print(array)
print(array2)

[1, 2, 3, 5, 6, 7, 9]
[1, 2, 3, 4, 5, 6, 7, 8, 10, 14]


## Zadanie 7

In [3]:
from matplotlib import pyplot as plt
import matplotlib.animation as animation

def insert_sort_anim(arr):
    """Animowanie sortowania bąbelkowego"""
    fig, ax = plt.subplots()
    n= len(arr)
    def update(frame):
        """Sortowanie bąbelkowe"""
        i = frame%(n-1)
        ax.clear()
        if arr[i] > arr[i+1]:
            arr[i],arr[i+1] = arr[i+1], arr[i]
        
        ax.bar(range(0,len(arr)),arr)


    ani = animation.FuncAnimation(fig, update, frames= (n-1)* n, repeat=False)
    ani.save("animacja.gif")
    plt.close()
    print(arr)


y = [7,2,6,10,4,14,8,3,1,5]
insert_sort_anim(y)





MovieWriter ffmpeg unavailable; using Pillow instead.


[1, 2, 3, 4, 5, 6, 7, 8, 10, 14]
